# Prepare subsets from CSV

## Import and Functions

In [1]:
import csv
import time
import winsound
import pathlib
import pandas as pd

def get_header_and_lookup_values(file: str):
    t = pd.read_csv(file, dtype=str)
    filter_column=t.columns[0]
    wanted_values = t[filter_column].to_list()
    return filter_column, wanted_values

def filter_source_file(source_file: str, filter_column: str, wanted_values: list, chunksize=10**5):
    print('Looking for mathces in column {} inside file:{}'.format(filter_column, source_file))
    filtered_data = pd.DataFrame(index=None)
    with pd.read_csv(source_file, chunksize=chunksize, dtype=str) as reader:
        for chunk in reader:
            to_be_added = chunk.loc[chunk[filter_column].isin(wanted_values)]
            filtered_data = pd.concat([filtered_data, to_be_added])
            print('Rows in filtered data: {}'.format(filtered_data.shape[0]))
    print('done reading {}'.format(source_file))
    return filtered_data

def write_df_to_csv(df, file_full_path: str, str_addition='_subset'):
    full_path = pathlib.Path(file_full_path)
    file_name = full_path.stem
    subset_full_path = "output/" + file_name + str_addition + '.csv'
    print('Writing: {}..'.format(subset_full_path),end='')
    df.to_csv(subset_full_path, index=False)
    print('done')
    



## Used files

In [2]:
#Training:
# YTMC_file = 'data/2022-08-17_YTMC_oilseed_SUBSET.csv'
# NFMC_file = 'data/2022-08-17_NFMC_oilseed_SUBSET.csv'
# INVB_file = 'data/2022-08-17_INVB_oilseed_SUBSET.csv'
# Real Oןilseeds:
# YTMC_file = 'C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/oil_seeds_170822/2022-08-17_YTMC_oilseed.csv'
# NFMC_file = 'C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/oil_seeds_170822/2022-08-17_NFMC_oilseed.csv'
# INVB_file = 'C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/oil_seeds_170822/2022-08-17_INVB_oilseed.csv'
# Real Wheat:
YTMC_file = 'C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/Wheat2022_10_03/2022-08-16_YTMC_wheat.csv'
NFMC_file = 'C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/Wheat2022_10_03/2022-10-03_NFMC_wheat.csv'
INVB_file = 'C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/Wheat2022_10_03/2022-08-16_INVB_wheat.csv'
# wanted_YTMC_values_file= 'Oilseeds_YTMC_doc_ids.csv'
# wanted_NFMC_values_file= 'Oilseeds_NFMC_doc_ids.csv'

# Main

In [3]:
startTime = time.time()
needed_pd_BAYER_UID = []

## YTMC

### Get header for filtering and values

In [4]:
# filter_column, wanted_values = get_header_and_lookup_values(wanted_YTMC_values_file)
filter_column = 'C_SET_PROJECT' #'C_SET_SEASON'
wanted_values = ['EIN:England','AIN:Ghent (Astene Greenhouse)','DHY:Discovery Breeding','GWW:Gatersleben Winter Wheat','ASW:Agronomic Services Wheat','GDB:Gaters Discovery Breeding','BWW:Milly la Foret Winter Whea','MWW:Mironovka Winter Wheat','AHY:Ghent (Astene Greenhouse)','BDB:Milly Discovery Breeding']

### read source and get df of the subset

In [5]:
df = filter_source_file(YTMC_file, filter_column, wanted_values)

Looking for mathces in column C_SET_PROJECT inside file:C:/Users/amnon/Desktop/ACCOUNTS/BASF/WS3_migration/Crop_zip_files/Wheat2022_10_03/2022-08-16_YTMC_wheat.csv
Rows in filtered data: 30146
Rows in filtered data: 65003
Rows in filtered data: 86501
Rows in filtered data: 110472
Rows in filtered data: 151686
Rows in filtered data: 182782
Rows in filtered data: 214265
Rows in filtered data: 243162
Rows in filtered data: 261021
Rows in filtered data: 298525


KeyboardInterrupt: 

### write subset to csv

In [ ]:
write_df_to_csv(df, YTMC_file)

### Add PD_BAYER_UID values to list

In [ ]:
needed_pd_BAYER_UID += [item for item in df['C_PD_BAYER_UID'].unique().tolist() if item]

## NFMC

### Get header for filtering and values

In [ ]:
# filter_column, wanted_values = get_header_and_lookup_values(wanted_NFMC_values_file)
# filter_column = 'C_SET_SEASON'
# wanted_values = ['22Q4']

### read source and get df of the subset

In [ ]:
df = filter_source_file(NFMC_file, filter_column, wanted_values)

### Write subset to csv

In [ ]:
write_df_to_csv(df, NFMC_file)

### Add PD_BAYER_UID values to list

In [ ]:
needed_pd_BAYER_UID += [item for item in df['C_PD_BAYER_UID'].unique().tolist() if item]

## INVB file by list

In [ ]:
# remove blanks from list
# needed_pd_BAYER_UID = YTMC_pd_BAYER_UID + NFMC_pd_BAYER_UID
# Create subset from another file (INVB)
    # Read source (INVB) and get df of the subset
invb_df = filter_source_file(INVB_file, 'C_PD_BAYER_UID', needed_pd_BAYER_UID)
    # write subset to CSV
write_df_to_csv(invb_df,INVB_file, '_subset')

## Finish

In [ ]:
winsound.Beep(440, 1000)
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))